### First Multiple Layered Perceptron (aka. Deep Learning Model)

<img src="images/single_neuron.png" alt="single_neuron" style="width:60%">

<img src="images/neural_network.png" alt="nn" style="width:90%">

<img src="images/deepnetwork.png" alt="dn" style="width:90%">

<img src="images/backpropa.png" alt="bp" style="width:60%">

### Human Brain Neuron System

"<table><tr><td><img src="images/brain.jpeg" alt="brain" style="width:100%"></td><td><img src="images/neurons.jpg" alt="nerons" style="width:50%"></td></tr></table>"

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np

##### Scikit Learn modules needed for Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder,MinMaxScaler , StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
def prepare_inputData(df):
    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_features = ['Age', 'Fare']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    categorical_features = ['Embarked', 'Sex', 'Pclass','Parch']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    # preprocess the input variable
    X = preprocessor.fit_transform(df[['Fare','Pclass', 'Name', 'Sex', 'Age','Embarked','Parch']])
    
    y_oneHot = OneHotEncoder()
    y = y_oneHot.fit_transform(df["Survived"])
    
    # split data for training and testing
    X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.2,random_state =42)
    
    return X_train, X_test, y_train, y_test

In [9]:
def train_model(X_train, X_test, y_train, y_test):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #print summary of model architecture
    model.summary()
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, y_train, epochs=10, batch_size=10)
    # evaluate the model
    scores = model.evaluate(X_test, y_test)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    return model

In [ ]:
X_train, X_test, y_train, y_test = prepare_inputData(df)
model = train_model(X_train, X_test, y_train, y_test)

In [18]:
y = df["Survived"].values
y = y.reshape(-1,1)
# split data for training and testing
#X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.2,random_state =42)

In [19]:
y.shape

(891, 1)

In [ ]:
test_data = pd.read_csv("test.csv")
X_test = test_data[['Fare','Pclass', 'Parch', 'Sex', 'Age','Embarked']]
X_submission = preprocessor.fit_transform(X_test)
X_submission.shape